In [ ]:
import sys
import mkl
mkl.get_max_threads()

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import LinearSVC 
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
from skimage.feature import hog
import cv2
from glob import glob
import time
import os
import pipeline as pl
import pickle
from sklearn.model_selection import train_test_split

from keras.preprocessing import image
from keras.models import load_model, Model, Sequential
from keras.layers import Input, Dense, Activation, Dropout, Flatten

import tensorflow as tf
import numpy as np
from keras.layers import Dense, Dropout, Flatten, Lambda, Input, Concatenate, Add, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

import pickle

# Reload custum module everytime
%load_ext autoreload
%aimport pipeline
%autoreload 1

## Loading Data Sets

In [ ]:
mldata_path = '/home/timo/Documents/mldata/project_5/'    
dataset = pl.get_image_files(mldata_path)
print(len(dataset['files']))
print(len(dataset['labels']))
dataset['images'] = []
for idx in range(0, len(dataset['files'])):
    dataset['images'].append(mpimg.imread(dataset['files'][idx]))
    

In [ ]:
print("Image Value: ", dataset['images'][0][0,0,0])
print(type(dataset['images'][0][0,0,0]))
print(type(dataset['images'][0]))
pl.print_image_properties(dataset['images'][0])

In [ ]:
X,y = pl.generate_batch_debug(1500, dataset, 'train')

## Model Design

In [ ]:
batch_size = 256
nb_epochs  = 20
steps_per_training_epoch = 100
steps_per_validation_epoch = np.max((1, int(0.1 * steps_per_training_epoch)) )
init = Input(shape=(64, 64, 3))
# Convolutional layers
x    = Conv2D(16, (3,3), activation='relu', padding='same', strides=(2,2) )(init)
x    = Conv2D(32, (3,3), activation='relu', padding='same', strides=(1,1) )(x)
x    = Dropout(0.4)(x)
x    = Conv2D(32, (3,3), activation='relu', padding='same', strides=(2,2) )(x)
x    = Conv2D(64, (3,3), activation='relu', padding='same', strides=(2,2) )(x)
x    = Dropout(0.6)(x)
x    = Conv2D(1, (8,8), activation='sigmoid', padding='valid', strides=(1,1) )(x)
out    = Flatten()(x)   
model = Model(init, out)

model.summary()
print("Set up training configuration ...")


optimizer = Adam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
training_batch_generator = pl.generate_batch(batch_size,
                                              dataset,
                                              'train')

validation_batch_generator = pl.generate_batch(batch_size,
                                                dataset,
                                                'validation')

print("Start training the model ...")
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    history = model.fit_generator(training_batch_generator,
                                  steps_per_epoch=steps_per_training_epoch,
                                  epochs=nb_epochs,
                                  validation_data=validation_batch_generator,
                                  validation_steps=steps_per_validation_epoch,
                                  verbose=1)


model.save('/home/timo/Documents/mldata/model.h5')

## Training Results

In [ ]:
print(history.history.keys())
plt.semilogy(history.history['loss'], 'r')
plt.semilogy(history.history['val_loss'])
plt.grid('on')
plt.show()

plt.plot(history.history['loss'], 'r')
plt.plot(history.history['val_loss'])
plt.grid('on')
plt.show()